Referência: https://towardsdatascience.com/preprocessing-text-data-using-python-576206753c28

In [1]:
import pandas as pd
import numpy as np
import nltk
import string
# import fasttext
import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

import matplotlib as plt

In [2]:
df = pd.read_csv('../Dados/steam_reviews_filtrado.csv')
# df = pd.read_csv('../Dados/steam_reviews.csv')

In [3]:
df.head()

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title,polaridade,review_len,word_count
0,2019-02-10,2,4,578,False,Recommended,Played as German Reich Declare war on Belgium...,Expansion - Hearts of Iron IV: Man the Guns,-0.016667,180,30
1,2019-02-10,0,0,184,False,Recommended,yes.,Expansion - Hearts of Iron IV: Man the Guns,0.000000,4,1
2,2019-02-07,0,0,892,False,Recommended,Very good game although a bit overpriced in my...,Expansion - Hearts of Iron IV: Man the Guns,0.162000,171,29
3,2018-06-14,126,1086,676,False,Recommended,Out of all the reviews I wrote This one is pro...,Dead by Daylight,0.078241,2094,419
4,2017-06-20,85,2139,612,False,Recommended,Disclaimer I survivor main. I play games for f...,Dead by Daylight,-0.114074,1397,273


In [4]:
for col in df.columns:
    print(col, df[col].isnull().sum())

date_posted 0
funny 0
helpful 0
hour_played 0
is_early_access_review 0
recommendation 0
review 8
title 0
polaridade 0
review_len 0
word_count 0


In [5]:
df.loc[df['review'].isnull()]

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title,polaridade,review_len,word_count
19376,2017-06-27,0,0,568,False,Not Recommended,NaN,Dead by Daylight,0.0,0,0
21988,2018-10-05,0,0,165,False,Recommended,NaN,MONSTER HUNTER: WORLD,0.0,0,0
34655,2018-11-24,0,0,85,False,Recommended,NaN,MONSTER HUNTER: WORLD,0.0,0,0
90059,2016-11-27,0,0,68,False,Recommended,NaN,Rocket League®,0.0,0,0
92345,2017-12-20,0,0,1070,False,Recommended,NaN,Rocket League®,0.0,0,0
246648,2017-09-18,2,1,728,True,Not Recommended,NaN,PLAYERUNKNOWN'S BATTLEGROUNDS,0.0,0,0
250449,2018-08-13,0,0,15,False,Not Recommended,NaN,PLAYERUNKNOWN'S BATTLEGROUNDS,0.0,0,0
294113,2018-01-19,0,0,558,False,Not Recommended,NaN,PLAYERUNKNOWN'S BATTLEGROUNDS,0.0,0,0


In [6]:
df = df.loc[df['review'].notnull()]

In [7]:
# rws = df.loc[:, ['recommendation', 'review']]
rws = df.copy()
rws

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title,polaridade,review_len,word_count
0,2019-02-10,2,4,578,False,Recommended,Played as German Reich Declare war on Belgium...,Expansion - Hearts of Iron IV: Man the Guns,-0.016667,180,30
1,2019-02-10,0,0,184,False,Recommended,yes.,Expansion - Hearts of Iron IV: Man the Guns,0.000000,4,1
2,2019-02-07,0,0,892,False,Recommended,Very good game although a bit overpriced in my...,Expansion - Hearts of Iron IV: Man the Guns,0.162000,171,29
3,2018-06-14,126,1086,676,False,Recommended,Out of all the reviews I wrote This one is pro...,Dead by Daylight,0.078241,2094,419
4,2017-06-20,85,2139,612,False,Recommended,Disclaimer I survivor main. I play games for f...,Dead by Daylight,-0.114074,1397,273
...,...,...,...,...,...,...,...,...,...,...,...
399899,2018-11-17,1,37,10,False,Recommended,YOUR FLESH WILL ROT AND DECAY.STEEL IS IMMORTA...,"Warhammer 40,000: Mechanicus",-0.034908,1977,341
399900,2018-11-17,3,41,38,False,Recommended,Domini and Dominae I believe what we are deali...,"Warhammer 40,000: Mechanicus",0.069079,2532,454
399901,2018-11-20,0,0,36,False,Recommended,First off if you like X Com style of games you...,"Warhammer 40,000: Mechanicus",0.090530,730,142
399902,2018-11-18,1,44,12,False,Recommended,As a disclaimer I'm an AdMech player on the ta...,"Warhammer 40,000: Mechanicus",0.042175,1789,336


In [8]:
rws['review'] = rws['review'].astype(str)

In [9]:
#90 segundos
def fix_contractions(text):
    words = []
    for word in text.split():
        try:
            words.append(contractions.fix(word))
        except:
            print('Error:', word, 'in', text)
            words.append(word)
    return words

rws['no_contract'] = rws['review'].apply(fix_contractions)

In [10]:
rws

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title,polaridade,review_len,word_count,no_contract
0,2019-02-10,2,4,578,False,Recommended,Played as German Reich Declare war on Belgium...,Expansion - Hearts of Iron IV: Man the Guns,-0.016667,180,30,"[Played, as, German, Reich, Declare, war, on, ..."
1,2019-02-10,0,0,184,False,Recommended,yes.,Expansion - Hearts of Iron IV: Man the Guns,0.000000,4,1,[yes.]
2,2019-02-07,0,0,892,False,Recommended,Very good game although a bit overpriced in my...,Expansion - Hearts of Iron IV: Man the Guns,0.162000,171,29,"[Very, good, game, although, a, bit, overprice..."
3,2018-06-14,126,1086,676,False,Recommended,Out of all the reviews I wrote This one is pro...,Dead by Daylight,0.078241,2094,419,"[Out, of, all, the, reviews, I, wrote, This, o..."
4,2017-06-20,85,2139,612,False,Recommended,Disclaimer I survivor main. I play games for f...,Dead by Daylight,-0.114074,1397,273,"[Disclaimer, I, survivor, main., I, play, game..."
...,...,...,...,...,...,...,...,...,...,...,...,...
399899,2018-11-17,1,37,10,False,Recommended,YOUR FLESH WILL ROT AND DECAY.STEEL IS IMMORTA...,"Warhammer 40,000: Mechanicus",-0.034908,1977,341,"[YOUR, FLESH, WILL, ROT, AND, DECAY.STEEL, IS,..."
399900,2018-11-17,3,41,38,False,Recommended,Domini and Dominae I believe what we are deali...,"Warhammer 40,000: Mechanicus",0.069079,2532,454,"[Domini, and, Dominae, I, believe, what, we, a..."
399901,2018-11-20,0,0,36,False,Recommended,First off if you like X Com style of games you...,"Warhammer 40,000: Mechanicus",0.090530,730,142,"[First, off, if, you, like, X, Com, style, of,..."
399902,2018-11-18,1,44,12,False,Recommended,As a disclaimer I'm an AdMech player on the ta...,"Warhammer 40,000: Mechanicus",0.042175,1789,336,"[As, a, disclaimer, I am, an, AdMech, player, ..."


In [11]:
rws['rating_description_str'] = [' '.join(map(str, l)) for l in rws['no_contract']]
rws

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title,polaridade,review_len,word_count,no_contract,rating_description_str
0,2019-02-10,2,4,578,False,Recommended,Played as German Reich Declare war on Belgium...,Expansion - Hearts of Iron IV: Man the Guns,-0.016667,180,30,"[Played, as, German, Reich, Declare, war, on, ...",Played as German Reich Declare war on Belgium ...
1,2019-02-10,0,0,184,False,Recommended,yes.,Expansion - Hearts of Iron IV: Man the Guns,0.000000,4,1,[yes.],yes.
2,2019-02-07,0,0,892,False,Recommended,Very good game although a bit overpriced in my...,Expansion - Hearts of Iron IV: Man the Guns,0.162000,171,29,"[Very, good, game, although, a, bit, overprice...",Very good game although a bit overpriced in my...
3,2018-06-14,126,1086,676,False,Recommended,Out of all the reviews I wrote This one is pro...,Dead by Daylight,0.078241,2094,419,"[Out, of, all, the, reviews, I, wrote, This, o...",Out of all the reviews I wrote This one is pro...
4,2017-06-20,85,2139,612,False,Recommended,Disclaimer I survivor main. I play games for f...,Dead by Daylight,-0.114074,1397,273,"[Disclaimer, I, survivor, main., I, play, game...",Disclaimer I survivor main. I play games for f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399899,2018-11-17,1,37,10,False,Recommended,YOUR FLESH WILL ROT AND DECAY.STEEL IS IMMORTA...,"Warhammer 40,000: Mechanicus",-0.034908,1977,341,"[YOUR, FLESH, WILL, ROT, AND, DECAY.STEEL, IS,...",YOUR FLESH WILL ROT AND DECAY.STEEL IS IMMORTA...
399900,2018-11-17,3,41,38,False,Recommended,Domini and Dominae I believe what we are deali...,"Warhammer 40,000: Mechanicus",0.069079,2532,454,"[Domini, and, Dominae, I, believe, what, we, a...",Domini and Dominae I believe what we are deali...
399901,2018-11-20,0,0,36,False,Recommended,First off if you like X Com style of games you...,"Warhammer 40,000: Mechanicus",0.090530,730,142,"[First, off, if, you, like, X, Com, style, of,...",First off if you like X Com style of games you...
399902,2018-11-18,1,44,12,False,Recommended,As a disclaimer I'm an AdMech player on the ta...,"Warhammer 40,000: Mechanicus",0.042175,1789,336,"[As, a, disclaimer, I am, an, AdMech, player, ...",As a disclaimer I am an AdMech player on the t...


In [ ]:
#para identificar linguas, contar qnts não forem 'en', e possivelmente removê-las
# Já foi realizado, os dados em inglês estão filtrados no 'steam_reviews_filtrado.csv'.

# pretrained_model = "lid.176.bin" 
# model = fasttext.load_model(pretrained_model)
# langs = []
# for sent in rws['rating_description_str']:
#     lang = model.predict(sent)[0]
#     langs.append(str(lang)[11:13])
# rws['langs'] = langs

In [12]:
#4min
#tokenizer
rws['tokenized'] = rws['rating_description_str'].apply(word_tokenize)
rws.head()

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title,polaridade,review_len,word_count,no_contract,rating_description_str,tokenized
0,2019-02-10,2,4,578,False,Recommended,Played as German Reich Declare war on Belgium...,Expansion - Hearts of Iron IV: Man the Guns,-0.016667,180,30,"[Played, as, German, Reich, Declare, war, on, ...",Played as German Reich Declare war on Belgium ...,"[Played, as, German, Reich, Declare, war, on, ..."
1,2019-02-10,0,0,184,False,Recommended,yes.,Expansion - Hearts of Iron IV: Man the Guns,0.000000,4,1,[yes.],yes.,"[yes, .]"
2,2019-02-07,0,0,892,False,Recommended,Very good game although a bit overpriced in my...,Expansion - Hearts of Iron IV: Man the Guns,0.162000,171,29,"[Very, good, game, although, a, bit, overprice...",Very good game although a bit overpriced in my...,"[Very, good, game, although, a, bit, overprice..."
3,2018-06-14,126,1086,676,False,Recommended,Out of all the reviews I wrote This one is pro...,Dead by Daylight,0.078241,2094,419,"[Out, of, all, the, reviews, I, wrote, This, o...",Out of all the reviews I wrote This one is pro...,"[Out, of, all, the, reviews, I, wrote, This, o..."
4,2017-06-20,85,2139,612,False,Recommended,Disclaimer I survivor main. I play games for f...,Dead by Daylight,-0.114074,1397,273,"[Disclaimer, I, survivor, main., I, play, game...",Disclaimer I survivor main. I play games for f...,"[Disclaimer, I, survivor, main, ., I, play, ga..."


In [13]:
#all lower
rws['lower'] = rws['tokenized'].apply(lambda x: [word.lower() for word in x])
rws.head()

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title,polaridade,review_len,word_count,no_contract,rating_description_str,tokenized,lower
0,2019-02-10,2,4,578,False,Recommended,Played as German Reich Declare war on Belgium...,Expansion - Hearts of Iron IV: Man the Guns,-0.016667,180,30,"[Played, as, German, Reich, Declare, war, on, ...",Played as German Reich Declare war on Belgium ...,"[Played, as, German, Reich, Declare, war, on, ...","[played, as, german, reich, declare, war, on, ..."
1,2019-02-10,0,0,184,False,Recommended,yes.,Expansion - Hearts of Iron IV: Man the Guns,0.000000,4,1,[yes.],yes.,"[yes, .]","[yes, .]"
2,2019-02-07,0,0,892,False,Recommended,Very good game although a bit overpriced in my...,Expansion - Hearts of Iron IV: Man the Guns,0.162000,171,29,"[Very, good, game, although, a, bit, overprice...",Very good game although a bit overpriced in my...,"[Very, good, game, although, a, bit, overprice...","[very, good, game, although, a, bit, overprice..."
3,2018-06-14,126,1086,676,False,Recommended,Out of all the reviews I wrote This one is pro...,Dead by Daylight,0.078241,2094,419,"[Out, of, all, the, reviews, I, wrote, This, o...",Out of all the reviews I wrote This one is pro...,"[Out, of, all, the, reviews, I, wrote, This, o...","[out, of, all, the, reviews, i, wrote, this, o..."
4,2017-06-20,85,2139,612,False,Recommended,Disclaimer I survivor main. I play games for f...,Dead by Daylight,-0.114074,1397,273,"[Disclaimer, I, survivor, main., I, play, game...",Disclaimer I survivor main. I play games for f...,"[Disclaimer, I, survivor, main, ., I, play, ga...","[disclaimer, i, survivor, main, ., i, play, ga..."


In [14]:
#remove punctuation
punc = string.punctuation
rws['no_punc'] = rws['lower'].apply(lambda x: [word for word in x if word not in punc])
rws.head()

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title,polaridade,review_len,word_count,no_contract,rating_description_str,tokenized,lower,no_punc
0,2019-02-10,2,4,578,False,Recommended,Played as German Reich Declare war on Belgium...,Expansion - Hearts of Iron IV: Man the Guns,-0.016667,180,30,"[Played, as, German, Reich, Declare, war, on, ...",Played as German Reich Declare war on Belgium ...,"[Played, as, German, Reich, Declare, war, on, ...","[played, as, german, reich, declare, war, on, ...","[played, as, german, reich, declare, war, on, ..."
1,2019-02-10,0,0,184,False,Recommended,yes.,Expansion - Hearts of Iron IV: Man the Guns,0.000000,4,1,[yes.],yes.,"[yes, .]","[yes, .]",[yes]
2,2019-02-07,0,0,892,False,Recommended,Very good game although a bit overpriced in my...,Expansion - Hearts of Iron IV: Man the Guns,0.162000,171,29,"[Very, good, game, although, a, bit, overprice...",Very good game although a bit overpriced in my...,"[Very, good, game, although, a, bit, overprice...","[very, good, game, although, a, bit, overprice...","[very, good, game, although, a, bit, overprice..."
3,2018-06-14,126,1086,676,False,Recommended,Out of all the reviews I wrote This one is pro...,Dead by Daylight,0.078241,2094,419,"[Out, of, all, the, reviews, I, wrote, This, o...",Out of all the reviews I wrote This one is pro...,"[Out, of, all, the, reviews, I, wrote, This, o...","[out, of, all, the, reviews, i, wrote, this, o...","[out, of, all, the, reviews, i, wrote, this, o..."
4,2017-06-20,85,2139,612,False,Recommended,Disclaimer I survivor main. I play games for f...,Dead by Daylight,-0.114074,1397,273,"[Disclaimer, I, survivor, main., I, play, game...",Disclaimer I survivor main. I play games for f...,"[Disclaimer, I, survivor, main, ., I, play, ga...","[disclaimer, i, survivor, main, ., i, play, ga...","[disclaimer, i, survivor, main, i, play, games..."


In [15]:
#remove stopwords
stop_words = set(stopwords.words('english'))
rws['stopwords_removed'] = rws['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])
rws.head()

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title,polaridade,review_len,word_count,no_contract,rating_description_str,tokenized,lower,no_punc,stopwords_removed
0,2019-02-10,2,4,578,False,Recommended,Played as German Reich Declare war on Belgium...,Expansion - Hearts of Iron IV: Man the Guns,-0.016667,180,30,"[Played, as, German, Reich, Declare, war, on, ...",Played as German Reich Declare war on Belgium ...,"[Played, as, German, Reich, Declare, war, on, ...","[played, as, german, reich, declare, war, on, ...","[played, as, german, reich, declare, war, on, ...","[played, german, reich, declare, war, belgium,..."
1,2019-02-10,0,0,184,False,Recommended,yes.,Expansion - Hearts of Iron IV: Man the Guns,0.000000,4,1,[yes.],yes.,"[yes, .]","[yes, .]",[yes],[yes]
2,2019-02-07,0,0,892,False,Recommended,Very good game although a bit overpriced in my...,Expansion - Hearts of Iron IV: Man the Guns,0.162000,171,29,"[Very, good, game, although, a, bit, overprice...",Very good game although a bit overpriced in my...,"[Very, good, game, although, a, bit, overprice...","[very, good, game, although, a, bit, overprice...","[very, good, game, although, a, bit, overprice...","[good, game, although, bit, overpriced, opinio..."
3,2018-06-14,126,1086,676,False,Recommended,Out of all the reviews I wrote This one is pro...,Dead by Daylight,0.078241,2094,419,"[Out, of, all, the, reviews, I, wrote, This, o...",Out of all the reviews I wrote This one is pro...,"[Out, of, all, the, reviews, I, wrote, This, o...","[out, of, all, the, reviews, i, wrote, this, o...","[out, of, all, the, reviews, i, wrote, this, o...","[reviews, wrote, one, probably, serious, one, ..."
4,2017-06-20,85,2139,612,False,Recommended,Disclaimer I survivor main. I play games for f...,Dead by Daylight,-0.114074,1397,273,"[Disclaimer, I, survivor, main., I, play, game...",Disclaimer I survivor main. I play games for f...,"[Disclaimer, I, survivor, main, ., I, play, ga...","[disclaimer, i, survivor, main, ., i, play, ga...","[disclaimer, i, survivor, main, i, play, games...","[disclaimer, survivor, main, play, games, fun,..."


### Lemmatization

In [16]:
#20min
#POS tags
rws['pos_tags'] = rws['stopwords_removed'].apply(nltk.tag.pos_tag)
rws.head()

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title,polaridade,review_len,word_count,no_contract,rating_description_str,tokenized,lower,no_punc,stopwords_removed,pos_tags
0,2019-02-10,2,4,578,False,Recommended,Played as German Reich Declare war on Belgium...,Expansion - Hearts of Iron IV: Man the Guns,-0.016667,180,30,"[Played, as, German, Reich, Declare, war, on, ...",Played as German Reich Declare war on Belgium ...,"[Played, as, German, Reich, Declare, war, on, ...","[played, as, german, reich, declare, war, on, ...","[played, as, german, reich, declare, war, on, ...","[played, german, reich, declare, war, belgium,...","[(played, JJ), (german, JJ), (reich, JJ), (dec..."
1,2019-02-10,0,0,184,False,Recommended,yes.,Expansion - Hearts of Iron IV: Man the Guns,0.000000,4,1,[yes.],yes.,"[yes, .]","[yes, .]",[yes],[yes],"[(yes, NNS)]"
2,2019-02-07,0,0,892,False,Recommended,Very good game although a bit overpriced in my...,Expansion - Hearts of Iron IV: Man the Guns,0.162000,171,29,"[Very, good, game, although, a, bit, overprice...",Very good game although a bit overpriced in my...,"[Very, good, game, although, a, bit, overprice...","[very, good, game, although, a, bit, overprice...","[very, good, game, although, a, bit, overprice...","[good, game, although, bit, overpriced, opinio...","[(good, JJ), (game, NN), (although, IN), (bit,..."
3,2018-06-14,126,1086,676,False,Recommended,Out of all the reviews I wrote This one is pro...,Dead by Daylight,0.078241,2094,419,"[Out, of, all, the, reviews, I, wrote, This, o...",Out of all the reviews I wrote This one is pro...,"[Out, of, all, the, reviews, I, wrote, This, o...","[out, of, all, the, reviews, i, wrote, this, o...","[out, of, all, the, reviews, i, wrote, this, o...","[reviews, wrote, one, probably, serious, one, ...","[(reviews, NNS), (wrote, VBD), (one, CD), (pro..."
4,2017-06-20,85,2139,612,False,Recommended,Disclaimer I survivor main. I play games for f...,Dead by Daylight,-0.114074,1397,273,"[Disclaimer, I, survivor, main., I, play, game...",Disclaimer I survivor main. I play games for f...,"[Disclaimer, I, survivor, main, ., I, play, ga...","[disclaimer, i, survivor, main, ., i, play, ga...","[disclaimer, i, survivor, main, i, play, games...","[disclaimer, survivor, main, play, games, fun,...","[(disclaimer, NN), (survivor, NN), (main, JJ),..."


In [17]:
#converte POS pro formato wordnet

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
rws['wordnet_pos'] = rws['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
rws.head()

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title,polaridade,review_len,word_count,no_contract,rating_description_str,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos
0,2019-02-10,2,4,578,False,Recommended,Played as German Reich Declare war on Belgium...,Expansion - Hearts of Iron IV: Man the Guns,-0.016667,180,30,"[Played, as, German, Reich, Declare, war, on, ...",Played as German Reich Declare war on Belgium ...,"[Played, as, German, Reich, Declare, war, on, ...","[played, as, german, reich, declare, war, on, ...","[played, as, german, reich, declare, war, on, ...","[played, german, reich, declare, war, belgium,...","[(played, JJ), (german, JJ), (reich, JJ), (dec...","[(played, a), (german, a), (reich, a), (declar..."
1,2019-02-10,0,0,184,False,Recommended,yes.,Expansion - Hearts of Iron IV: Man the Guns,0.000000,4,1,[yes.],yes.,"[yes, .]","[yes, .]",[yes],[yes],"[(yes, NNS)]","[(yes, n)]"
2,2019-02-07,0,0,892,False,Recommended,Very good game although a bit overpriced in my...,Expansion - Hearts of Iron IV: Man the Guns,0.162000,171,29,"[Very, good, game, although, a, bit, overprice...",Very good game although a bit overpriced in my...,"[Very, good, game, although, a, bit, overprice...","[very, good, game, although, a, bit, overprice...","[very, good, game, although, a, bit, overprice...","[good, game, although, bit, overpriced, opinio...","[(good, JJ), (game, NN), (although, IN), (bit,...","[(good, a), (game, n), (although, n), (bit, n)..."
3,2018-06-14,126,1086,676,False,Recommended,Out of all the reviews I wrote This one is pro...,Dead by Daylight,0.078241,2094,419,"[Out, of, all, the, reviews, I, wrote, This, o...",Out of all the reviews I wrote This one is pro...,"[Out, of, all, the, reviews, I, wrote, This, o...","[out, of, all, the, reviews, i, wrote, this, o...","[out, of, all, the, reviews, i, wrote, this, o...","[reviews, wrote, one, probably, serious, one, ...","[(reviews, NNS), (wrote, VBD), (one, CD), (pro...","[(reviews, n), (wrote, v), (one, n), (probably..."
4,2017-06-20,85,2139,612,False,Recommended,Disclaimer I survivor main. I play games for f...,Dead by Daylight,-0.114074,1397,273,"[Disclaimer, I, survivor, main., I, play, game...",Disclaimer I survivor main. I play games for f...,"[Disclaimer, I, survivor, main, ., I, play, ga...","[disclaimer, i, survivor, main, ., i, play, ga...","[disclaimer, i, survivor, main, i, play, games...","[disclaimer, survivor, main, play, games, fun,...","[(disclaimer, NN), (survivor, NN), (main, JJ),...","[(disclaimer, n), (survivor, n), (main, a), (p..."


In [18]:
#lematização em si
# acontecem algum erros, como 'played' n fica 'play', provavelmente devido a inacurácia da POS tag.
#90segs
wnl = WordNetLemmatizer()
rws['lemmatized'] = rws['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
rws.head()

,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title,polaridade,review_len,word_count,no_contract,rating_description_str,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,2019-02-10,2,4,578,False,Recommended,Played as German Reich Declare war on Belgium...,Expansion - Hearts of Iron IV: Man the Guns,-0.016667,180,30,"[Played, as, German, Reich, Declare, war, on, ...",Played as German Reich Declare war on Belgium ...,"[Played, as, German, Reich, Declare, war, on, ...","[played, as, german, reich, declare, war, on, ...","[played, as, german, reich, declare, war, on, ...","[played, german, reich, declare, war, belgium,...","[(played, JJ), (german, JJ), (reich, JJ), (dec...","[(played, a), (german, a), (reich, a), (declar...","[played, german, reich, declare, war, belgium,..."
1,2019-02-10,0,0,184,False,Recommended,yes.,Expansion - Hearts of Iron IV: Man the Guns,0.000000,4,1,[yes.],yes.,"[yes, .]","[yes, .]",[yes],[yes],"[(yes, NNS)]","[(yes, n)]",[yes]
2,2019-02-07,0,0,892,False,Recommended,Very good game although a bit overpriced in my...,Expansion - Hearts of Iron IV: Man the Guns,0.162000,171,29,"[Very, good, game, although, a, bit, overprice...",Very good game although a bit overpriced in my...,"[Very, good, game, although, a, bit, overprice...","[very, good, game, although, a, bit, overprice...","[very, good, game, although, a, bit, overprice...","[good, game, although, bit, overpriced, opinio...","[(good, JJ), (game, NN), (although, IN), (bit,...","[(good, a), (game, n), (although, n), (bit, n)...","[good, game, although, bit, overpriced, opinio..."
3,2018-06-14,126,1086,676,False,Recommended,Out of all the reviews I wrote This one is pro...,Dead by Daylight,0.078241,2094,419,"[Out, of, all, the, reviews, I, wrote, This, o...",Out of all the reviews I wrote This one is pro...,"[Out, of, all, the, reviews, I, wrote, This, o...","[out, of, all, the, reviews, i, wrote, this, o...","[out, of, all, the, reviews, i, wrote, this, o...","[reviews, wrote, one, probably, serious, one, ...","[(reviews, NNS), (wrote, VBD), (one, CD), (pro...","[(reviews, n), (wrote, v), (one, n), (probably...","[review, write, one, probably, serious, one, w..."
4,2017-06-20,85,2139,612,False,Recommended,Disclaimer I survivor main. I play games for f...,Dead by Daylight,-0.114074,1397,273,"[Disclaimer, I, survivor, main., I, play, game...",Disclaimer I survivor main. I play games for f...,"[Disclaimer, I, survivor, main, ., I, play, ga...","[disclaimer, i, survivor, main, ., i, play, ga...","[disclaimer, i, survivor, main, i, play, games...","[disclaimer, survivor, main, play, games, fun,...","[(disclaimer, NN), (survivor, NN), (main, JJ),...","[(disclaimer, n), (survivor, n), (main, a), (p...","[disclaimer, survivor, main, play, game, fun, ..."


In [22]:
rws = rws.loc[rws['lemmatized'].apply(lambda x: len(x) != 0)]

In [23]:
# rws.to_csv('../Dados/steam_reviews_lemmatization.csv', index=False)